In [42]:
import pandas as pd
import numpy as np
import json
from difflib import get_close_matches

In [137]:
EDU_SYS_CONVERTER = {
    "博士" : "博士班",
    "碩士" : "碩士班",
    "碩專" : "碩士在職專班",
    "大學" : "學士班"
}

class table29(object):
    def __init__(self, configure):
        self.configure = configure
        self.data = []
        self.number = 0
    def append(self, value):
        self.data.append(value)
        self.number += value["student_sum"]
    
    def outputMainTable(self):
        self.configure["曾修讀程式設計課程人數"] = self.number
        return self.configure
    
def constructureDataStructure(config):
    ds = {}
    depts_names = list(set(config.單位))
    for dept_name in depts_names:
        ndfs = config[config["單位"] == dept_name]
        ds[dept_name] = {}
        eduSyses = list(set(ndfs.學制班別))
        for eduSys in eduSyses:
            ds[dept_name][eduSys] = table29(ndfs.loc[ndfs["學制班別"] == eduSys])
            
    return ds

def classify(dept, dept_lists):
    mostLikely = get_close_matches(dept, dept_lists)
    return mostLikely[0]

**Read Config data**

In [151]:
config = pd.read_excel("./11003_29 修讀程式設計課程情形.xls")
configDS = constructureDataStructure(config)

In [136]:
phd_depts = list(set(config[config["學制班別"] == "博士班"].單位))
master_depts = list(set(config[config["學制班別"] == "碩士班"].單位))
master_depts2 = list(set(config[config["學制班別"] == "碩士在職專班"].單位))
undergraduate_depts = list(set(config[config["學制班別"] == "學士班"].單位))

EDU_SYS_MAPPING = {
    "博士" : phd_depts,
    "碩士" : master_depts,
    "碩專" : master_depts2,
    "大學" : undergraduate_depts
}

In [140]:
data = pd.read_excel("./學29﹍1090319.xls")
data["tmp_dept_name"] = np.full(data.shape[0], np.nan)
data["tmp_dept_name"] = data["dept_name"].str.replace(r'碩、\w+', '')
data["tmp_dept_name"] = data["tmp_dept_name"].str.replace(r'應用數學', '')

In [143]:
# try to open old dept_map.json data. 
# If not found, creat it!!!. But DO NOT FORGET TO CHECK THE OUTPUT DATA
try: 
    with open("./dept_maps.json", "r", encoding='utf') as file:
        txt = file.read()
        dept_maps = json.loads(txt)
except:
    dept_maps = {}
    for i in range(len(data.values)):
        dept_name = data.at[i, "tmp_dept_name"]
        try:
            data.at[i, "tmp_dept_name"] = classify(data.at[i, "tmp_dept_name"] , EDU_SYS_MAPPING[data.at[i, "acad"]])
        except IndexError:
            print("ERROR occure when classifing ",dept_name)
            raise IndexError
    #     print(dept_name, "->", data.at[i, "tmp_dept_name"])
        dept_maps[data["dept_name"][i] + "_" + data.at[i, "acad"]] = data["tmp_dept_name"][i]

    with open("dept_maps.json", "w") as file:
        file.write(json.dumps(dept_maps, indent=4, ensure_ascii=False))

In [145]:
for i in range(len(data.values)):
    dept_name = dept_maps[data.at[i, "dept_name"] + "_" + data.at[i, "acad"]]
    eduSys = EDU_SYS_CONVERTER[data.at[i, "acad"]]
    try:
        configDS[dept_name][eduSys].append(data.loc[i])
    except:
        print(i, data.at[i, "dept_name"], " -> ", dept_name, "acad = ", eduSys)
        raise KeyError

In [146]:
ndata = []
for i in configDS:
    for j in configDS[i]:
        ndata.append(configDS[i][j].outputMainTable())
ndataArray = np.array(ndata)

In [148]:
temp = []
for i in ndata:
    temp.append(i.values[0])

In [149]:
result = pd.DataFrame(temp, columns=config.columns)

In [150]:
result.to_excel("output29.xlsx")